# CellChat

In [49]:
options(warn=-1, stringsAsFactors=FALSE)

In [50]:
library_load <- suppressMessages(
    
    list(
        
        # CellTalk
        library(CellChat), 
        
        # Seurat 
        library(Seurat), 
        
        # Data 
        library(tidyverse), 
        
        # Plotting 
        library(ComplexHeatmap), 
        library(circlize), 
        library(viridis), 
        library(ggplotify), 
        library(ggrepel), 
        library(cowplot)
        
    )
)

In [51]:
random_seed <- 42
set.seed(random_seed)

In [52]:
ht_opt$message=FALSE # ComplexHeatmap 

In [53]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [54]:
# Source files
source("plotting_global.R")
source("bin/cell_type.R")
source("bin/cellchat.R")

In [55]:
# Plotting Theme
ggplot2::theme_set(theme_global_set()) # From project global source()

# Parameter and files 

In [61]:
cnt_file <- "data/BSA_0885_FACS_FD/ANALYSIS/cnt.rds"
meta_file <- "data/BSA_0885_FACS_FD/ANALYSIS/meta.rds"

In [62]:
cache <- FALSE

# Import 

In [65]:
meta <- readRDS(meta_file)
cnt <- readRDS(cnt_file)
cnt <- cnt[, !colnames(cnt) %in% c("mgi_symbol", "ensembl_gene_id")]
cnt <- cnt[rowSums(cnt>=5)>=3, ]

[1] 28876   107

## Pre-process cellchat objects for each treatment condition 

In [ ]:
cellchat_pp <- function(cnt, meta, group_by) {
    
    cellchat <- createCellChat(object=as.matrix(cnt), meta=meta, group.by=group_by)
    cellchat <- setIdent(cellchat, ident.use=group_by)
    cellchat@DB <- CellChatDB.mouse # Add DB slot with interaction, complex, cofactor, and geneInfo data frames 
    cellchat <- subsetData(cellchat) # Subset data for signaling genes 
    cellchat <- identifyOverExpressedGenes(cellchat, thresh.p=1) # Identify DE signaling genes stored in object@var.features. Disabled for now since I want to keep all genes for analysis. 
    cellchat <- identifyOverExpressedInteractions(cellchat) # Filter interaction based on over expressed genes. Adds @LR$LRsig which is a pruned persion of cellchat@DB$interaction
    cellchat <- projectData(cellchat, PPI.mouse) # Project protein data 
    cellchat <- computeCommunProb(cellchat, raw.use=FALSE, type="truncatedMean", trim=0.01) # Using the information from DB compute interaction strength (probability) in object@net$prob and associated p-values in object@net$pval. https://github.com/sqjin/CellChat/issues/137
    cellchat <- computeCommunProbPathway(cellchat) # Summarise L-R interaction on pathway level. Return object@netP$prob is the pathway probability and significant pathways are stored in object@netP$pathways
    cellchat <- aggregateNet(cellchat) # Updates object@net with counts and weights 
    cellchat <- netAnalysis_computeCentrality(cellchat)
    
    return(cellchat)
    
}

In [ ]:
if(!cache) {
    
    cellchat_1 <- cellchat_pp(cnt[, meta$genotype=="WT" & meta$dpi=="D0"], meta[meta$genotype=="WT" & meta$dpi=="D0", ], group_by="celltype")
    saveRDS(cellchat_1, "data/BSA_0885_FACS_FD/ANALYSIS/cellchat/wt_d0.rds")

} else {
    
    cellchat_1 <- readRDS("data/BSA_0885_FACS_FD/ANALYSIS/cellchat/wt_d0.rds")
    
}

In [ ]:
if(!cache) {
    
    cellchat_2 <- cellchat_pp(cnt[, meta$genotype=="WT" & meta$dpi=="D1"], meta[meta$genotype=="WT" & meta$dpi=="D1", ], group_by="celltype")
    saveRDS(cellchat_2, "data/BSA_0885_FACS_FD/ANALYSIS/cellchat/wt_d1.rds")

} else {
    
    cellchat_2 <- readRDS("data/BSA_0885_FACS_FD/ANALYSIS/cellchat/wt_d1.rds")
    
}

In [ ]:
if(!cache) {
    
    cellchat_3 <- cellchat_pp(cnt[, meta$genotype=="WT" & meta$dpi=="D3"], meta[meta$genotype=="WT" & meta$dpi=="D3", ], group_by="celltype")
    saveRDS(cellchat_3, "data/BSA_0885_FACS_FD/ANALYSIS/cellchat/wt_d3.rds")

} else {
    
    cellchat_3 <- readRDS("data/BSA_0885_FACS_FD/ANALYSIS/cellchat/wt_d3.rds")
    
}

In [ ]:
if(!cache) {
    
    cellchat_4 <- cellchat_pp(cnt[, meta$genotype=="WT" & meta$dpi=="D7"], meta[meta$genotype=="WT" & meta$dpi=="D7", ], group_by="celltype")
    saveRDS(cellchat_4, "data/BSA_0885_FACS_FD/ANALYSIS/cellchat/wt_d7.rds")

} else {
    
    cellchat_4 <- readRDS("data/BSA_0885_FACS_FD/ANALYSIS/cellchat/wt_d7.rds")
    
}

In [ ]:
if(!cache) {
    
    cellchat_5 <- cellchat_pp(cnt[, meta$genotype=="IFNARKO" & meta$dpi=="D0"], meta[meta$genotype=="IFNARKO" & meta$dpi=="D0", ], group_by="celltype")
    saveRDS(cellchat_5, "data/BSA_0885_FACS_FD/ANALYSIS/cellchat/ifnarko_d0.rds")

} else {
    
    cellchat_5 <- readRDS("data/BSA_0885_FACS_FD/ANALYSIS/cellchat/ifnarko_d0.rds")
    
}

In [ ]:
if(!cache) {
    
    cellchat_6 <- cellchat_pp(cnt[, meta$genotype=="IFNARKO" & meta$dpi=="D1"], meta[meta$genotype=="IFNARKO" & meta$dpi=="D1", ], group_by="celltype")
    saveRDS(cellchat_6, "data/BSA_0885_FACS_FD/ANALYSIS/cellchat/ifnarko_d1.rds")

} else {
    
    cellchat_6 <- readRDS("data/BSA_0885_FACS_FD/ANALYSIS/cellchat/ifnarko_d1.rds")
    
}

In [ ]:
if(!cache) {
    
    cellchat_7 <- cellchat_pp(cnt[, meta$genotype=="IFNARKO" & meta$dpi=="D3"], meta[meta$genotype=="IFNARKO" & meta$dpi=="D3", ], group_by="celltype")
    saveRDS(cellchat_7, "data/BSA_0885_FACS_FD/ANALYSIS/cellchat/ifnarko_d3.rds")

} else {
    
    cellchat_7 <- readRDS("data/BSA_0885_FACS_FD/ANALYSIS/cellchat/ifnarko_d3.rds")
    
}

In [ ]:
if(!cache) {
    
    cellchat_8 <- cellchat_pp(cnt[, meta$genotype=="IFNARKO" & meta$dpi=="D7"], meta[meta$genotype=="IFNARKO" & meta$dpi=="D7", ], group_by="celltype")
    saveRDS(cellchat_8, "data/BSA_0885_FACS_FD/ANALYSIS/cellchat/ifnarko_d7.rds")

} else {
    
    cellchat_8 <- readRDS("data/BSA_0885_FACS_FD/ANALYSIS/cellchat/ifnarko_d7.rds")
    
}

# Merge CellChat data objects 

In [ ]:
cellchat_diff_1 <- list(WT=cellchat_1, IFNARKO=cellchat_5)
cellchat_diff_1 <- mergeCellChat(cellchat_diff_1, add.names=names(cellchat_diff_1))

In [ ]:
cellchat_diff_2 <- list(WT=cellchat_2, IFNARKO=cellchat_6)
cellchat_diff_2 <- mergeCellChat(cellchat_diff_2, add.names=names(cellchat_diff_2))

In [ ]:
cellchat_diff_3 <- list(WT=cellchat_3, IFNARKO=cellchat_7)
cellchat_diff_3 <- mergeCellChat(cellchat_diff_3, add.names=names(cellchat_diff_3))

In [ ]:
cellchat_diff_4 <- list(WT=cellchat_4, IFNARKO=cellchat_8)
cellchat_diff_4 <- mergeCellChat(cellchat_diff_4, add.names=names(cellchat_diff_4))

# Heatmap of number of interactions 

In [ ]:
options(repr.plot.width=20, repr.plot.height=7.5)

hm_1 <- hm_net(cellchat=cellchat_1, slot="count", title="Interaction count\nWT D0") %>% draw() %>% grid.grabExpr()
hm_2 <- hm_net(cellchat=cellchat_2, slot="count", title="Interaction count\nWT D1") %>% draw() %>% grid.grabExpr()
hm_3 <- hm_net(cellchat=cellchat_3, slot="count", title="Interaction count\nWT D3") %>% draw() %>% grid.grabExpr()
hm_4 <- hm_net(cellchat=cellchat_4, slot="count", title="Interaction count\nWT D7") %>% draw() %>% grid.grabExpr()

hm_5 <- hm_net(cellchat=cellchat_5, slot="count", title="Interaction count\nIFNARKO D0") %>% draw() %>% grid.grabExpr()
hm_6 <- hm_net(cellchat=cellchat_6, slot="count", title="Interaction count\nIFNARKO D1") %>% draw() %>% grid.grabExpr()
hm_7 <- hm_net(cellchat=cellchat_7, slot="count", title="Interaction count\nIFNARKO D3") %>% draw() %>% grid.grabExpr()
hm_8 <- hm_net(cellchat=cellchat_8, slot="count", title="Interaction count\nIFNARKO D7") %>% draw() %>% grid.grabExpr()

patchwork::wrap_plots(list(hm_1, hm_2, hm_3, hm_4, hm_5, hm_6, hm_7, hm_8), ncol=4)

In [ ]:
options(repr.plot.width=20, repr.plot.height=7.5)

hm_1 <- hm_net(cellchat=cellchat_diff_1, slot="count", title="Interaction count (D0)\nWT vs IFNARKO", diff=TRUE) %>% draw() %>% grid.grabExpr()
hm_2 <- hm_net(cellchat=cellchat_diff_2, slot="count", title="Interaction count (D1)\nWT vs IFNARKO", diff=TRUE) %>% draw() %>% grid.grabExpr()
hm_3 <- hm_net(cellchat=cellchat_diff_3, slot="count", title="Interaction count (D3)\nWT vs IFNARKO", diff=TRUE) %>% draw() %>% grid.grabExpr()
hm_4 <- hm_net(cellchat=cellchat_diff_4, slot="count", title="Interaction count (D7)\nWT vs IFNARKO", diff=TRUE) %>% draw() %>% grid.grabExpr()

patchwork::wrap_plots(list(hm_1, hm_2, hm_3, hm_4), ncol=4)

# Net analysis all cell types between groups

In [ ]:
plot_1 <- rankNet(cellchat_diff_1, mode="comparison", stacked=TRUE, do.stat=TRUE) + theme_global_set(1) + ggtitle("D0\nWT vs INFARKO") + scale_fill_manual(values=color$genotype)
plot_2 <- rankNet(cellchat_diff_2, mode="comparison", stacked=TRUE, do.stat=TRUE) + theme_global_set(1) + ggtitle("D1\nWT vs INFARKO") + scale_fill_manual(values=color$genotype)
plot_3 <- rankNet(cellchat_diff_3, mode="comparison", stacked=TRUE, do.stat=TRUE) + theme_global_set(1) + ggtitle("D3\nWT vs INFARKO") + scale_fill_manual(values=color$genotype)
plot_4 <- rankNet(cellchat_diff_4, mode="comparison", stacked=TRUE, do.stat=TRUE) + theme_global_set(1) + ggtitle("D7\nWT vs INFARKO") + scale_fill_manual(values=color$genotype)

In [ ]:
options(repr.plot.width=4*6, repr.plot.height=40)

ggpubr::ggarrange(plotlist=list(plot_1, plot_2, plot_3, plot_4), ncol=4, nrow=1, common.legend=FALSE, legend="bottom")

# Signaling strength

In [ ]:
options(repr.plot.width=4*5, repr.plot.height=10)

object_list <- list(cellchat_1, cellchat_2, cellchat_3, cellchat_4, cellchat_5, cellchat_6, cellchat_7, cellchat_8)
names(object_list) <- c("WT D0", "WT D1", "WT D3", "WT D7", "IFNARKO D0", "IFNARKO D1", "IFNARKO D3", "IFNARKO D7")

num_link <- sapply(object_list, function(x) {rowSums(x@net$count) + colSums(x@net$count)-diag(x@net$count)})
weight_lim <- c(min(unlist(num_link)), max(unlist(num_link))) # control the dot size in the different datasets

net_plot_list <- list()
for (i in names(object_list)) {
    
    data <- netAnalysis_signalingRole_scatter(object_list[[i]], title=names(object_list)[i], weight.MinMax=weight_lim)$data
    
    net_plot <- ggplot(data, aes(x=x, y=y, label=labels, size=Count, color=labels)) + 
        geom_point() + xlim(0, 1) + ylim(0, 1) + 
        geom_text_repel(segment.color="black", force=20, force_pull=1, max.overlaps=getOption("ggrepel.max.overlaps", default=100), size=5, alpha=1, guide="none", segment.size=0.1, color="black", parse=TRUE) + 
        scale_color_manual(values=color$cell_type_main, na.value="grey50", labels=base::str2expression(names(color$cell_type_main))) + 
        guides(colour=guide_legend(override.aes=list(size=3))) + 
        ggtitle(i) + xlab("Outgoing interaction strength") + ylab("Incoming interaction strength") + 
        theme_global_set(1) + theme(aspect.ratio=1)
    
    
    net_plot_list[[i]] <- net_plot

}

ggpubr::ggarrange(plotlist=net_plot_list, ncol=4, nrow=2, common.legend=TRUE, legend="bottom")

# Net analysis outgoing/incoming signals per cell type between groups

In [ ]:
suppressMessages(plot_1 <- lapply(levels(cellchat_diff_1@idents[[3]]), function(ident) {netAnalysis_signalingChanges_scatter(cellchat_diff_1, idents.use=ident, label.size=4, dot.size=5, color.use=c("#9C9C9C", color$genotype[[1]], color$genotype[[2]])) + ggtitle(parse(text=ident)) + theme_global_set(size=2)}))
suppressMessages(plot_2 <- lapply(levels(cellchat_diff_2@idents[[3]]), function(ident) {netAnalysis_signalingChanges_scatter(cellchat_diff_2, idents.use=ident, label.size=4, dot.size=5, color.use=c("#9C9C9C", color$genotype[[1]], color$genotype[[2]])) + ggtitle(parse(text=ident)) + theme_global_set(size=2)}))
suppressMessages(plot_3 <- lapply(levels(cellchat_diff_3@idents[[3]]), function(ident) {netAnalysis_signalingChanges_scatter(cellchat_diff_3, idents.use=ident, label.size=4, dot.size=5, color.use=c("#9C9C9C", color$genotype[[1]], color$genotype[[2]])) + ggtitle(parse(text=ident)) + theme_global_set(size=2)}))
suppressMessages(plot_4 <- lapply(levels(cellchat_diff_4@idents[[3]]), function(ident) {netAnalysis_signalingChanges_scatter(cellchat_diff_4, idents.use=ident, label.size=4, dot.size=5, color.use=c("#9C9C9C", color$genotype[[1]], color$genotype[[2]])) + ggtitle(parse(text=ident)) + theme_global_set(size=2)}))

In [ ]:
options(repr.plot.width=4*5, repr.plot.height=ceiling(length(plot_1)/4)*5)

ggpubr::ggarrange(plotlist=plot_1, ncol=4, nrow=ceiling(length(plot_1)/4), common.legend=TRUE, legend="bottom")
ggpubr::ggarrange(plotlist=plot_2, ncol=4, nrow=ceiling(length(plot_2)/4), common.legend=TRUE, legend="bottom")
ggpubr::ggarrange(plotlist=plot_3, ncol=4, nrow=ceiling(length(plot_3)/4), common.legend=TRUE, legend="bottom")
ggpubr::ggarrange(plotlist=plot_4, ncol=4, nrow=ceiling(length(plot_4)/4), common.legend=TRUE, legend="bottom")

# Net analysis of significant interactions enriched in baseline compared to GVHD

In [ ]:
options(repr.plot.width=4*5, repr.plot.height=80)

plot_1 <- suppressMessages(lapply(levels(cellchat_diff_1@idents[[3]]), function(i) {
    
    tryCatch(
        
        {
            netVisual_bubble(cellchat_diff_1, sources.use=i, comparison=c(1, 2), remove.isolate=TRUE, thresh=0.01, title.name=parse(text=i), color.text=color$genotype[c(1, 2)], font.size.title=20, font.size=16)
        
        }, error=function(e) {
            
            NULL
        
        }
    
    )

}
                                 )
                          
                          )

ggpubr::ggarrange(plotlist=plot_1[!sapply(plot_1, is.null)], ncol=4, nrow=1, common.legend=FALSE, legend="bottom")

In [ ]:
options(repr.plot.width=4*5, repr.plot.height=80)

plot_1 <- suppressMessages(lapply(levels(cellchat_diff_2@idents[[3]]), function(i) {
    
    tryCatch(
        
        {
            netVisual_bubble(cellchat_diff_2, sources.use=i, comparison=c(1, 2), remove.isolate=TRUE, thresh=0.01, title.name=parse(text=i), color.text=color$genotype[c(1, 2)], font.size.title=20, font.size=16)
        
        }, error=function(e) {
            
            NULL
        
        }
    
    )

}
                                 )
                          
                          )

ggpubr::ggarrange(plotlist=plot_1[!sapply(plot_1, is.null)], ncol=4, nrow=1, common.legend=FALSE, legend="bottom")

In [ ]:
options(repr.plot.width=4*5, repr.plot.height=80)

plot_1 <- suppressMessages(lapply(levels(cellchat_diff_3@idents[[3]]), function(i) {
    
    tryCatch(
        
        {
            netVisual_bubble(cellchat_diff_3, sources.use=i, comparison=c(1, 2), remove.isolate=TRUE, thresh=0.01, title.name=parse(text=i), color.text=color$genotype[c(1, 2)], font.size.title=20, font.size=16)
        
        }, error=function(e) {
            
            NULL
        
        }
    
    )

}
                                 )
                          
                          )

ggpubr::ggarrange(plotlist=plot_1[!sapply(plot_1, is.null)], ncol=4, nrow=1, common.legend=FALSE, legend="bottom")

In [ ]:
options(repr.plot.width=4*5, repr.plot.height=80)

plot_1 <- suppressMessages(lapply(levels(cellchat_diff_4@idents[[3]]), function(i) {
    
    tryCatch(
        
        {
            netVisual_bubble(cellchat_diff_4, sources.use=i, comparison=c(1, 2), remove.isolate=TRUE, thresh=0.01, title.name=parse(text=i), color.text=color$genotype[c(1, 2)], font.size.title=20, font.size=16)
        
        }, error=function(e) {
            
            NULL
        
        }
    
    )

}
                                 )
                          
                          )

ggpubr::ggarrange(plotlist=plot_1[!sapply(plot_1, is.null)], ncol=4, nrow=1, common.legend=FALSE, legend="bottom")

# CCR2 ACKR1 

In [ ]:
# parilr <- data.frame(interaction_name=c("CCL2_ACKR1", "CCL5_ACKR1", "CCL17_ACKR1"))

In [ ]:
# options(repr.plot.width=5, repr.plot.height=5)

# plot_1 <- netVisual_chord_gene(cellchat_1, lab.cex=1, color.use=unlist(color$cell_type_main), pairLR.use=parilr, sources.use = c(1, 2, 3), targets.use = NULL, thresh=0.05, show.legend=FALSE, small.gap=3)
# plot_2 <- netVisual_chord_gene(cellchat_2, lab.cex=1, color.use=unlist(color$cell_type_main), pairLR.use=parilr, sources.use = c(1, 2, 3), targets.use = 4, thresh=0.05, show.legend=FALSE, small.gap=3)
# plot_3 <- netVisual_chord_gene(cellchat_3, lab.cex=1, color.use=unlist(color$cell_type_main), pairLR.use=parilr, sources.use = c(1, 2, 3), targets.use = 4, thresh=0.05, show.legend=FALSE, small.gap=3)
# plot_4 <- netVisual_chord_gene(cellchat_4, lab.cex=1, color.use=unlist(color$cell_type_main), pairLR.use=parilr, sources.use = c(1, 2, 3), targets.use = 4, thresh=0.05, show.legend=FALSE, small.gap=3)
# plot_5 <- netVisual_chord_gene(cellchat_5, lab.cex=1, color.use=unlist(color$cell_type_main), pairLR.use=parilr, sources.use = c(1, 2, 3), targets.use = NULL, thresh=0.05, show.legend=FALSE, small.gap=3)
# plot_6 <- netVisual_chord_gene(cellchat_6, lab.cex=1, color.use=unlist(color$cell_type_main), pairLR.use=parilr, sources.use = c(1, 2, 3), targets.use = 4, thresh=0.05, show.legend=FALSE, small.gap=3)
# plot_7 <- netVisual_chord_gene(cellchat_7, lab.cex=1, color.use=unlist(color$cell_type_main), pairLR.use=parilr, sources.use = c(1, 2, 3), targets.use = 4, thresh=0.05, show.legend=FALSE, small.gap=3)
# plot_8 <- netVisual_chord_gene(cellchat_8, lab.cex=1, color.use=unlist(color$cell_type_main), pairLR.use=parilr, sources.use = c(1, 2, 3), targets.use = 4, thresh=0.05, show.legend=FALSE, small.gap=3)

# cowplot::plot_grid(plotlist=list(plot_1, plot_2), ncol=2, nrow=1)